# Imports

Brittany C. Haas and Melissa A. Hardy's jupyter notebook for automated collection of molecular descriptors and post-processing (i.e., Boltzmann average, min/max values, etc.).

**NOTE: Make sure to use the get_properties_environment file to set your conda environment.**

In [80]:
import os,re,sys,pickle,datetime,time,random,itertools,glob
import warnings
warnings.filterwarnings("ignore")
import openpyxl
import pandas as pd
from rdkit import Chem
import get_properties_functions as gp

# Atom Inputs Dataframe

Portions of this section were adapted from code written Jordan P. Liles.

## Generate dataframe with atom numbers

### Use command line to prepare files

To create files: navigate to folder that contains all the log files you wish to analyze.

    module load openbabel
    obabel *.log -osdf -m  
    ls *.log > log_ids.txt
    cat *.sdf >> molecules.sdf

You will use the log_ids.txt and molecules.sdf files in the rest of 2.1.

To create files: navigate to folder that contains all the log files you wish to analyze.

    

You will use the log_ids.txt and molecules.sdf files in the rest of 2.1.

### Define SMARTS substructure


Recommended to draw the common substructure (with general atoms) in Chemdraw and copy as SMILES (this will generate a SMARTS string)

More information about SMARTS and available characters here: https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html


In [52]:
#this is the common smarts substructure for the molecules you will analyze
#you have to explicitly draw hydrogens into the SMARTS structure if you want to collect properties for hydrogen atoms
substructure = Chem.MolFromSmarts('cBr')

### Generate preliminary dataframe

In [53]:
#generate a list of molecules using RDkit
all_compounds = Chem.SDMolSupplier('molecules.sdf', removeHs=False) 
#molecules.sdf is generated with the instructions above
#it is a single sdf that contains the structures/atom numbers etc. for every molecule you will analyze

#uses RDKit to search for the substructure in each compound you will analyze
atoms = []
for molecule in all_compounds:
    if molecule is not None:
        submatch = molecule.GetSubstructMatches(substructure) #find substructure
        matchlist = list([item for sublist in submatch for item in sublist]) #list of zero-indexed atom numbers
        match_idx = [x+1 for x in matchlist] #this line changes from 0-indexed to 1-indexed (for Gaussian)
        atoms.append(match_idx) #append 1-indexed list to atoms (a list of lists)
        
#this loop extracts log names from log_ids and splits them to the desired format
filenames = open("log_ids.txt", "r") #generate this with instruction above
#it is a text file that contains the file name for every molecule you will analyze
list_of_filenames = [(line.strip()).split() for line in filenames] #list of the file names (each of which includes all conformers)
list_of_files = []
for filename in list_of_filenames:
    file = filename[0].split(".")
    list_of_files.append(file[0])
filenames.close()

#put the atom numbers for the substructure for each log file into a dataframe
prelim_df = pd.DataFrame(atoms) 
index=list_of_files
prelim_df.insert(0,column='log_name',value=list_of_files)
display(prelim_df)

,log_name,0,1
0,smiles_00000_conf_0,NaN,NaN
1,smiles_00000_conf_1,NaN,NaN
2,smiles_00001_conf_0,5.0,6.0
3,smiles_00002_conf_0,6.0,7.0
4,smiles_00003_conf_0,NaN,NaN
...,...,...,...
298,smiles_00126_conf_0,9.0,10.0
299,smiles_00126_conf_1,9.0,10.0
300,smiles_00126_conf_2,9.0,10.0
301,smiles_00126_conf_3,9.0,10.0


### Define column headers using GaussView

Using the preliminary dataframe displayed above, open one of your files and check the atom numbers. 

Assign labels to each column using the cell below. You will call these column headers when you select your properties. 

**User input required:**

In [85]:
atom_labels = {'log_name': 'log_name',
                0: 'C4',
                1: 'C1',} 
              

### Generate labeled dataframe

**NOTE: it is very important you assign these correctly otherwise the properties you collect will be for the wrong atoms and not produce meaningful correlations.** 

We recommend checking the numbering/headers for at least two different compounds. 

Numbering for different conformers of the same compounds will likely be the same (but may not be for some symmetrical groups).

In [55]:
#rename columns using the user input above
atom_map_df = prelim_df.rename(columns=atom_labels)
display(atom_map_df)

#you can use this to clean up the table if you have more atoms in your substructure than you want to collect descriptors for
#atom_map_df = atom_map_df.drop(columns= ['C4', 'C1']) 
#display(atom_map_df.head())

df = atom_map_df #df is what properties will be appended to, this creates a copy so that you have the original preserved 

,log_name,C4,C1
0,smiles_00000_conf_0,NaN,NaN
1,smiles_00000_conf_1,NaN,NaN
2,smiles_00001_conf_0,5.0,6.0
3,smiles_00002_conf_0,6.0,7.0
4,smiles_00003_conf_0,NaN,NaN
...,...,...,...
298,smiles_00126_conf_0,9.0,10.0
299,smiles_00126_conf_1,9.0,10.0
300,smiles_00126_conf_2,9.0,10.0
301,smiles_00126_conf_3,9.0,10.0


### Save atom map to Excel (if desired)

In [56]:
writer = pd.ExcelWriter('test_2.xlsx')
atom_map_df.to_excel(writer)
writer.save()

## Import a manually-generated atom mapping dataframe

If you need to manually generate the atom mapping dataframe, check out the atom_map_sample.xlsx to make sure you have the desired format. 

In [57]:
atom_map_df = pd.read_excel('test_2.xlsx','Sheet1',index_col=0,header=0,engine='openpyxl')
display(atom_map_df.head())

,log_name,C4,C1
0,smiles_00000_conf_0,NaN,NaN
1,smiles_00000_conf_1,NaN,NaN
2,smiles_00001_conf_0,5.0,6.0
3,smiles_00002_conf_0,6.0,7.0
4,smiles_00003_conf_0,NaN,NaN


# Define Properties to Collect

## Available property functions and how to call them: 

In [58]:
#this box has functions to choose from
df = atom_map_df

#---------------GoodVibes Engergies---------------
#uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
#calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
#inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

#---------------Frontier Orbitals-----------------
#E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

#---------------Polarizability--------------------
#Exact polarizability
df = gp.get_polarizability(df)

#---------------Dipole----------------------------
#Total dipole moment magnitude in Debye
df = gp.get_dipole(df)

#---------------Volume----------------------------
#Molar volume
#requires the Gaussian keyword = "volume" in the .com file
df = gp.get_volume(df)

#---------------SASA------------------------------
#Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
df = gp.get_SASA(df)

#---------------NBO-------------------------------
#natural charge from NBO
#requires the Gaussian keyword = "pop=nbo7" in the .com file
nbo_list = ["C1", "O2", "O3", "C4"]
df = gp.get_nbo(df, nbo_list) 

#---------------NMR-------------------------------
#isotropic NMR shift
#requires the Gaussian keyword = "nmr=giao" in the .com file
nmr_list = ["C1", "C4", "H5"]
df = gp.get_nmr(df, nmr_list) 

#---------------Distance--------------------------
#distance between 2 atoms
dist_list_of_lists = [["O2", "C1"], ["O3", "H5"], ["C4", "C1"]]
df = gp.get_distance(df, dist_list_of_lists) 

#---------------Angle-----------------------------
#angle between 3 atoms
angle_list_of_lists = [["O3", "C1", "O2"], ["C4", "C1", "O3"]]
df = gp.get_angles(df, angle_list_of_lists) 

#---------------Dihedral--------------------------
#dihedral angle between 4 atoms
dihedral_list_of_lists = [["O2", "C1", "O3", "H5"], ["C4", "C1", "O3", "H5"]]
df = gp.get_dihedral(df, dihedral_list_of_lists) 

#---------------Vbur Scan-------------------------
#uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
#inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
#if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C4"]
df = gp.get_vbur_scan(df, vbur_list, 2, 4, 0.5)
    
#---------------Sterimol morfeus------------------
#uses morfeus to calculate Sterimol L, B1, and B5 values
#NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
sterimol_list_of_lists = [["O3", "H5"], ["C1", "C4"]]
df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists) 

#---------------Buried Sterimol-------------------
#uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
#atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
#for more information: https://kjelljorner.github.io/morfeus/sterimol.html
#inputs: dataframe, list of atom pairs, r_buried
sterimol_list_of_lists = [["C1", "C4"]]
df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5) 

#---------------Sterimol DBSTEP-------------------
#uses DBSTEP to calculate Sterimol L, B1, and B5 values
#default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
#more info here: https://github.com/patonlab/DBSTEP
#NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
sterimol_list_of_lists = [["O3", "H5"]]
df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists) 

#---------------Sterimol2Vec----------------------
#uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size 
#default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
#more info here: https://github.com/patonlab/DBSTEP
#inputs: dataframe, list of atom pairs, end_radius, and step_size
sterimol2vec_list_of_lists = [["C1", "C4"]]
df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0) 

#---------------Pyramidalization------------------
#uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
#collects values based on two definitions of pyramidalization
#details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
pyr_list = ["C1", "C4"]
df = gp.get_pyramidalization(df, pyr_list)

#---------------Plane Angle-----------------------
#plane angle between 2 planes (each defined by 3 atoms)
planeangle_list_of_lists = [["O2", "C1", "O3", "H5", "C1", "C4"], ["O2", "C1", "O3", "H5", "C1", "C4"]]
df = gp.get_planeangle(df, planeangle_list_of_lists) 

#---------------Time----------------------------------
#returns the total CPU time and total Wall time
#if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
df = gp.get_time(df)

#---------------ChelpG----------------------------
#ChelpG ESP charge 
#requires the Gaussian keyword = "pop=chelpg" in the .com file
a_list = ['C1']
df = gp.get_chelpg(df, a_list) 

#---------------Hirshfeld-------------------------
#Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
#requires the Gaussian keyword = "pop=hirshfeld" in the .com file
a_list = ['C1']
df = gp.get_hirshfeld(df, a_list) 

#---------------IR--------------------------------
#CAUTION: CANNOT ACCURATELY IDENTIFY ATOM STRETCHES IN SOME CASES (struggles if there is more than one stretch in the defined range)
#IR frequencies and intensities in a specific range (for specific atoms)
#requires the Gaussian keyword = "freq=noraman" in the .com file
#inputs: dataframe, atom1, atom2, frequency_min, frequency_max, intensity_min, intensity_max, threshold
#if you want to collect multiple IR frequencies, you will need to copy/paste this function for each stretch
#we recommend a threshold of 0.0 (may have to adjust)
df = gp.get_IR(df, "C1", "O2", 1700, 1900, 100, 800, 0.0)


pd.options.display.max_columns = None
display(df)



   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using v

,log_name,C4,C1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(au),polar_aniso(au),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_O2,NBO_charge_O3,NBO_charge_C4,NMR_shift_C1,NMR_shift_C4,NMR_shift_H5,%Vbur_C1_2.0Å,%Vbur_C4_2.0Å,%Vbur_C1_2.5Å,%Vbur_C4_2.5Å,%Vbur_C1_3.0Å,%Vbur_C4_3.0Å,%Vbur_C1_3.5Å,%Vbur_C4_3.5Å,%Vbur_C1_4.0Å,%Vbur_C4_4.0Å,Buried_Sterimol_L_C1_C4_5.0(Å),Buried_Sterimol_B1_C1_C4_5.0(Å),Buried_Sterimol_B5_C1_C4_5.0(Å),pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,pyramidalization_Gavrish_C4(°),pyramidalization_Agranat-Radhakrishnan_C4,CPU_time_total(hours),Wall_time_total(hours),ChelpG_charge_C1,Hirsh_charge_C1,Hirsh_CM5_charge_C1,Hirsh_atom_dipole_C1,IR_freq_C1_O2
0,smiles_00000_conf_0,NaN,NaN,-1016.571578,0.211837,-1016.346571,0.05303,0.051456,-1016.399601,-1016.398027,298.15,-0.30038,-0.01331,-0.156845,0.28707,0.04285,170.716,105.812,4.7787,1900.502,398.760207,602.554485,0.865173,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,1.85194,1.314472,no data,no data,no data,no data,no data
1,smiles_00000_conf_1,NaN,NaN,-1016.571578,0.211837,-1016.346571,0.05303,0.051456,-1016.3996,-1016.398026,298.15,-0.30037,-0.01332,-0.156845,0.28705,0.04285,170.719,105.825,4.7787,1742.101,398.660205,602.432069,0.865273,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,2.02767,1.337806,no data,no data,no data,no data,no data
2,smiles_00001_conf_0,5.0,6.0,-3297.515202,0.057214,-3297.450404,0.040574,0.040302,-3297.490978,-3297.490706,298.15,-0.32502,-0.03854,-0.18178,0.28648,0.05767,102.463,91.3258,2.8667,1114.114,282.581837,391.368455,0.915652,no data,no data,no data,no data,no data,no data,no data,34.129649,96.43595,30.147155,86.979099,27.206868,73.944987,24.376281,59.395383,21.403467,44.817206,no data,no data,no data,no data,no data,no data,no data,0.21357,0.144917,no data,no data,no data,no data,no data
3,smiles_00002_conf_0,6.0,7.0,-3048.558348,0.100486,-3048.448498,0.044343,0.044142,-3048.492841,-3048.492641,298.15,-0.32757,-0.04326,-0.185415,0.28431,0.06046,123.078,89.5195,4.1072,1427.972,313.729599,454.484539,0.911188,no data,no data,no data,no data,no data,no data,no data,34.323347,96.403667,31.241047,87.330707,29.126937,76.014973,27.066833,63.822563,24.634954,51.435355,no data,no data,no data,no data,no data,no data,no data,1.07522,0.819139,no data,no data,no data,no data,no data
4,smiles_00003_conf_0,NaN,NaN,-899.858131,0.145741,-899.701897,0.04653,0.045576,-899.748427,-899.747473,298.15,-0.27558,0.00619,-0.134695,0.28177,0.03219,135.574,109.729,5.2505,1562.681,345.050453,499.077447,0.88182,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,0.80489,0.608972,no data,no data,no data,no data,no data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,smiles_00126_conf_0,9.0,10.0,-3065.797687,0.109402,-3065.677286,0.048813,0.047842,-3065.7261,-3065.725129,298.15,-0.33361,-0.05888,-0.196245,0.27473,0.07009,122.031,80.9379,2.683,1216.681,338.680844,483.805726,0.879983,no data,no data,no data,no data,no data,no data,no data,33.642175,94.053461,29.30069,83.777185,25.630401,69.892543,22.167932,56.515561,19.023493,45.058888,no data,no data,no data,no data,no data,no data,no data,0.99442,0.778139,no data,no data,no data,no data,no data
299,smiles_00126_conf_1,9.0,10.0,-3065.797

## Copy and modify available property functions above to customize

We recommend copying the entire cell above. You will need to change the atom number lists to match your desired column headers and delete (or comment out) any properites you don't want to collect.

In [59]:
#this box has functions to choose from
df = atom_map_df

#---------------GoodVibes Engergies---------------
#uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
#calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
#inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

#---------------Frontier Orbitals-----------------
#E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

#---------------Vbur Scan-------------------------
#uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
#inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
#if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C4"]
df = gp.get_vbur_scan(df, vbur_list, 2, 4, 0.5)

pd.options.display.max_columns = None
display(df)



   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using v

,log_name,C4,C1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,%Vbur_C1_2.0Å,%Vbur_C4_2.0Å,%Vbur_C1_2.5Å,%Vbur_C4_2.5Å,%Vbur_C1_3.0Å,%Vbur_C4_3.0Å,%Vbur_C1_3.5Å,%Vbur_C4_3.5Å,%Vbur_C1_4.0Å,%Vbur_C4_4.0Å
0,smiles_00000_conf_0,NaN,NaN,-1016.571578,0.211837,-1016.346571,0.05303,0.051456,-1016.399601,-1016.398027,298.15,-0.30038,-0.01331,-0.156845,0.28707,0.04285,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data
1,smiles_00000_conf_1,NaN,NaN,-1016.571578,0.211837,-1016.346571,0.05303,0.051456,-1016.3996,-1016.398026,298.15,-0.30037,-0.01332,-0.156845,0.28705,0.04285,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data
2,smiles_00001_conf_0,5.0,6.0,-3297.515202,0.057214,-3297.450404,0.040574,0.040302,-3297.490978,-3297.490706,298.15,-0.32502,-0.03854,-0.18178,0.28648,0.05767,34.129649,96.43595,30.147155,86.979099,27.206868,73.944987,24.376281,59.395383,21.403467,44.817206
3,smiles_00002_conf_0,6.0,7.0,-3048.558348,0.100486,-3048.448498,0.044343,0.044142,-3048.492841,-3048.492641,298.15,-0.32757,-0.04326,-0.185415,0.28431,0.06046,34.323347,96.403667,31.241047,87.330707,29.126937,76.014973,27.066833,63.822563,24.634954,51.435355
4,smiles_00003_conf_0,NaN,NaN,-899.858131,0.145741,-899.701897,0.04653,0.045576,-899.748427,-899.747473,298.15,-0.27558,0.00619,-0.134695,0.28177,0.03219,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,smiles_00126_conf_0,9.0,10.0,-3065.797687,0.109402,-3065.677286,0.048813,0.047842,-3065.7261,-3065.725129,298.15,-0.33361,-0.05888,-0.196245,0.27473,0.07009,33.642175,94.053461,29.30069,83.777185,25.630401,69.892543,22.167932,56.515561,19.023493,45.058888
299,smiles_00126_conf_1,9.0,10.0,-3065.797694,0.109402,-3065.677293,0.048818,0.047845,-3065.726111,-3065.725138,298.15,-0.3336,-0.05885,-0.196225,0.27475,0.07007,33.638946,93.966296,29.277901,83.785324,25.640644,69.89813,22.149296,56.538273,19.007976,45.074794
300,smiles_00126_conf_2,9.0,10.0,-3065.795868,0.109288,-3065.675531,0.049128,0.048006,-3065.724659,-3065.723537,298.15,-0.33374,-0.05921,-0.196475,0.27453,0.07031,33.638946,93.963068,29.281156,83.808113,25.649024,69.869264,22.154538,56.411317,18.957545,44.901775
301,smiles_00126_conf_3,9.0,10.0,-3065.795868,0.109289,-3065.67553,0.049125,0.048005,-3065.724655,-3065.723535,298.15,-0.33374,-0.05921,-0.196475,0.27453,0.07031,33.645403,93.963068,29.303946,83.773929,25.657405,69.864608,22.162109,56.39501,18.955605,44.878111


## Save collected properties to Excel

Helpful to save here in case the Notebook crashes or if you want to add more properties before post-processsing. Can be read in at 5.1.1.

In [60]:
writer = pd.ExcelWriter('All_Conformer_Properties_example_test_2.xlsx')
df.to_excel(writer)
writer.save()

# Post-processing

## User input for data processing

In [70]:
#for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
#this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
prefix = "Ac" 
suffix = "_"

#columns that provide atom mapping information are dropped
atom_columns_to_drop = ["C1", "O2", "O3", "C4", "H5"]

#title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"


energy_cutoff = 4.2 #specify energy cutoff in kcal/mol to remove conformers above this value before post-processing
verbose = False #set to true if you'd like to see info on the nunmber of conformers removed for each compound

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

Check out the dataframe_sample.xlsx to make sure you have the desired format. 

In [82]:
df = pd.read_excel('All_Conformer_Properties_example_1.xlsx','Sheet1',index_col=0,header=0,engine='openpyxl')
display(df.head())

,log_name,C4,C1,E_spc (Hartree),G(T)_spc(Hartree),H_spc(Hartree),T,T*S,T*qh_S,ZPE(Hartree),qh_G(T)_spc(Hartree),HOMO,LUMO,η,μ,ω,%Vbur_C1_2.0Å,%Vbur_C4_2.0Å,%Vbur_C1_2.5Å,%Vbur_C4_2.5Å,%Vbur_C1_3.0Å,%Vbur_C4_3.0Å,%Vbur_C1_3.5Å,%Vbur_C4_3.5Å,%Vbur_C1_4.0Å,%Vbur_C4_4.0Å
0,Ac1_clust-1,3,2,-543.543441,-543.291813,-543.235970,298.15,0.055843,0.054099,0.292507,-543.290069,-0.35353,0.03938,0.39291,-0.157075,0.03140,96.871772,97.691761,90.127295,84.967118,78.622244,71.062090,65.065924,60.080251,54.098520,49.841723
1,Ac1_clust-10,3,2,-543.537866,-543.292382,-543.230502,298.15,0.061880,0.057455,0.291621,-543.287958,-0.35342,0.04394,0.39736,-0.154740,0.03013,94.473140,97.536803,82.479815,85.004558,65.471357,71.097475,48.637835,59.546217,37.247455,48.395118
2,Ac1_clust-11,3,2,-543.539014,-543.290972,-543.231499,298.15,0.059473,0.056151,0.292038,-543.287650,-0.35148,0.04371,0.39519,-0.153885,0.02996,94.521565,98.276085,82.623063,87.275361,65.543057,75.093117,49.114214,64.315831,38.336773,53.410325
3,Ac1_clust-12,3,2,-543.540547,-543.294717,-543.233299,298.15,0.061417,0.057290,0.291455,-543.290589,-0.35482,0.04101,0.39583,-0.156905,0.03110,96.064695,97.136493,85.509181,82.992577,69.311494,67.682881,52.907193,54.992080,40.855627,43.393489
4,Ac1_clust-13,3,2,-543.538204,-543.290149,-543.230894,298.15,0.059255,0.055999,0.291839,-543.286894,-0.35491,0.03949,0.39440,-0.157710,0.03153,95.454545,97.520661,86.689348,85.657312,72.240018,72.403903,56.592434,60.887416,44.779576,49.495686


## Generating a list of compounds that have conformational ensembles

**ONLY RUN THE AUTOMATED OR THE MANUAL CELL, NOT BOTH**

**AUTOMATED:** if your compounds are named consistenly, this section generates your compound list based on the similar naming structure

In [83]:
#this is a template for our files that are all named "AcXXX_clust-X.log"

compound_list = []
    
for index, row in df.iterrows():
    log_file = row['log_name'] #read file name from df
    prefix_and_compound = log_file.split(str(suffix)) #splits to get "AcXXX" (entry O) (and we don't use the "clust-X" (entry 1))
    compound = prefix_and_compound[0].split(str(prefix)) #splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
    compound_list.append(compound[1])

compound_list = list(set(compound_list)) #removes duplicate stuctures that result from having conformers of each
compound_list.sort() #reorders numerically (not sure if it reorders alphabetically)
print(compound_list)

#this should generate a list that looks like this: ['24', '27', '34', '48']

['1', '2', '3', '4', '5']


In [67]:
compound_list = []

for index, row in df.iterrows():
    log_file = row['log_name']  # read file name from df
    prefix_and_compound = log_file.split('_conf_')  # split on '_conf_' to get 'smiles_00000'
    compound = prefix_and_compound[0]  # take the part before '_conf_'
    last_digit = compound[-1]  # get the last character of the string
    compound_list.append(last_digit)

compound_list = list(set(compound_list))  # remove duplicate digits
compound_list.sort()  # reorder numerically
print(compound_list)



['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


**MANUAL:** if your comment naming scheme is not consistent or you have trouble with the template above, you can manually define your compound list

In [38]:
compound_list = [1, 2, 3, 4, 5]

## Post-processing to get properties for each compound

In [89]:
all_df_master = pd.DataFrame(columns=[])
properties_df_master = pd.DataFrame(columns=[])

for compound in compound_list: 
    #defines the common start to all files using the input above 
    substring = str(prefix) + str(compound) + str(suffix)
    
    #makes a data frame for one compound at a time for post-processing
    valuesdf = df[df["log_name"].str.startswith(substring)]
    valuesdf = valuesdf.drop(columns = atom_columns_to_drop)
    valuesdf = valuesdf.reset_index(drop = True)  #you must re-index otherwise the 2nd, 3rd, etc. compounds fail
   
    #define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
    non_boltz_columns = ["G(Hartree)","∆G(Hartree)","∆G(kcal/mol)", "e^(-∆G/RT)","Mole Fraction"] #don't boltzman average columns containing these strings in the column label
    reg_avg_columns = ['CPU_time_total(hours)', 'Wall_time_total(hours)'] #don't boltzmann average these either, we average them in case that is helpful
    gv_extra_columns = ['E_spc (Hartree)', 'H_spc(Hartree)', 'T', 'T*S', 'T*qh_S', 'ZPE(Hartree)', 'qh_G(T)_spc(Hartree)', "G(T)_spc(Hartree)"]
    gv_extra_columns.remove(str(energy_col_header))
    
    #calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
    valuesdf["∆G(Hartree)"] = valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
    valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
    valuesdf["e^(-∆G/RT)"] = np.exp((valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)) #R is in cal/(K*mol)
    valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
    initial = len(valuesdf.index)
    if verbose: 
        print(prefix + str(compound))
        #display(valuesdf)
        print("Total number of conformers = ", initial)
    valuesdf.drop(valuesdf[valuesdf["∆G(kcal/mol)"] >= energy_cutoff].index, inplace=True) #E cutoff applied here
    valuesdf = valuesdf.reset_index(drop = True) #resetting indexes
    final = len(valuesdf.index) 
    if verbose: 
        print("Number of conformers above ", energy_cutoff, " kcal/mol: ", initial-final)
    values_boltz_row = []
    values_min_row = []
    values_max_row = []
    values_boltz_stdev_row =[]
    values_range_row = []
    values_exclude_columns = []
    
    for column in valuesdf:
        if "log_name" in column:
            values_boltz_row.append("Boltzmann Averages")
            values_min_row.append("Ensemble Minimum")
            values_max_row.append("Ensemble Maximum")
            values_boltz_stdev_row.append("Boltzmann Standard Deviation")
            values_range_row.append("Ensemble Range")
            values_exclude_columns.append(column) #used later to build final dataframe
        elif any(phrase in column for phrase in non_boltz_columns) or any(phrase in column for phrase in gv_extra_columns):
            values_boltz_row.append("")
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        elif any(phrase in column for phrase in reg_avg_columns):
            values_boltz_row.append(valuesdf[column].mean()) #intended to print the average CPU/wall time in the boltz column
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        else:
            valuesdf[column] = pd.to_numeric(valuesdf[column]) #to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
            values_boltz_row.append((valuesdf[column] * valuesdf["Mole Fraction"]).sum())
            values_min_row.append(valuesdf[column].min())
            values_max_row.append(valuesdf[column].max())
            values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

            
            # this section generates the weighted std deviation (weighted by mole fraction) 
            # formula: https://www.statology.org/weighted-standard-deviation-excel/
    
            boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum() #number
            delta_values_sq = []
    
            #makes a list of the "deviation" for each conformer           
            for index, row in valuesdf.iterrows(): 
                value = row[column]
                delta_value_sq = (value - boltz)**2
                delta_values_sq.append(delta_value_sq)
            
            #w is list of weights (i.e. mole fractions)
            w = list(valuesdf["Mole Fraction"])
            wstdev = np.sqrt( (np.average(delta_values_sq, weights=w)) / (((len(w)-1)/len(w))*np.sum(w)) )
            if len(w) == 1: #if there is only one conformer in the ensemble, set the weighted standard deviation to 0 
                wstdev = 0
            #np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
            
            values_boltz_stdev_row.append(wstdev)
            
            
    valuesdf.loc[len(valuesdf)] = values_boltz_row
    valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
    valuesdf.loc[len(valuesdf)] = values_min_row
    valuesdf.loc[len(valuesdf)] = values_max_row
    valuesdf.loc[len(valuesdf)] = values_range_row

    #final output format is built here:
    explicit_order_front_columns = ["log_name", energy_col_header,"∆G(Hartree)","∆G(kcal/mol)","e^(-∆G/RT)","Mole Fraction"]
    
    #reorders the dataframe using front columns defined above
    valuesdf = valuesdf[explicit_order_front_columns + [col for col in valuesdf.columns if col not in explicit_order_front_columns and col not in values_exclude_columns]]
    
    #determine the index of the lowest energy conformer
    low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
    
    #copy the row to a new_row with the name of the log changed to Lowest E Conformer
    new_row = valuesdf.loc[low_e_index[0]]
    new_row['log_name'] = "Lowest E Conformer"   
    valuesdf =  valuesdf.append(new_row, ignore_index=True)

#------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------  
    #if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
    #this template can be adjusted for min/max/etc. 
    
    #find the index for the min or max column:
    ensemble_min_index = valuesdf[valuesdf["log_name"] == "Ensemble Minimum"].index.tolist()
    
    #find the min or max value of the property (based on index above)
    #saves the value in a list (min_value) with one entry (this is why we call min_value[0])
    min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C4_3.0Å"].tolist()   
    vbur_min_index = valuesdf[valuesdf["%Vbur_C4_3.0Å"] == min_value[0]].index.tolist()
    
    #copy the row to a new_row with the name of the log changed to Property_min_conformer
    new_row = valuesdf.loc[vbur_min_index[0]]
    new_row['log_name'] = "%Vbur_C4_3.0Å_min_Conformer"   
    valuesdf =  valuesdf.append(new_row, ignore_index=True)
#--------------------------------------------------------------------------------------------------------------------    
    
    #appends the frame to the master output
    all_df_master = pd.concat([all_df_master, valuesdf])
    
    #drop all the individual conformers
    dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
    valuesdf = valuesdf.drop(dropindex)
    valuesdf = valuesdf.reset_index(drop = True)
    
    #display(valuesdf)   
    
    #drop the columns created to determine the mole fraction and some that 
    valuesdf = valuesdf.drop(columns = explicit_order_front_columns)
    try:
        valuesdf = valuesdf.drop(columns = gv_extra_columns)
    except:
        pass
    try:
        valuesdf = valuesdf.drop(columns = reg_avg_columns)
    except:
        pass
        
#---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------  
    compound_name = prefix + str(compound) 
#--------------------------------------------------------------------------------------------------------------------      

    properties_df = pd.DataFrame({'Compound_Name': [compound_name]})
    
    #builds a dataframe (for each compound) by adding summary properties as new columns
    for (columnName, columnData) in valuesdf.iteritems():
        #the indexes need to match the values dataframe - display it to double check if you need to make changes 
        #(uncomment the display(valuesdf) in row 124 of this cell)
        properties_df[str(columnName) + "_Boltz"] = [columnData.values[0]]
        properties_df[str(columnName) + "_Boltz_stdev"] = [columnData.values[1]]
        properties_df[str(columnName) + "_min"] = [columnData.values[2]]
        properties_df[str(columnName) + "_max"] = [columnData.values[3]]
        properties_df[str(columnName) + "_range"] = [columnData.values[4]]
        properties_df[str(columnName) + "_low_E"] = [columnData.values[5]]
        
        #if you're collecting properties for a specific conformer, add these here (note the index)
        #example:
        properties_df[str(columnName) + "_V_bur_min"] = [columnData.values[6]]
        
        #if you only want a table with Boltz, you can comment out the other summary properties to generate a Boltz spreadsheet
        #of if you don't want to collect range, etc.
    #concatenates the individual acid properties df into the master properties df
    properties_df_master = pd.concat([properties_df_master, properties_df], axis = 0)

all_df_master = all_df_master.reset_index(drop = True)
properties_df_master = properties_df_master.reset_index(drop = True)


KeyError: "['O2', 'O3', 'H5'] not found in axis"

### Peek at your new dataframes

In [69]:
display(properties_df_master.head())
display(all_df_master)

""


""


### Save to Microsoft Excelᵀᴹ 

In [13]:
all_df_master.to_excel('All_Conformer_and_Summary_Properties_example.xlsx', index = False)
properties_df_master.to_excel('Summary_Properties_example.xlsx', index = False)